In [9]:
import torch
from torchvision.models.detection.mask_rcnn import resnet50
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import random_split
from tqdm import tqdm
from torcheval.metrics import MulticlassF1Score, MulticlassPrecision
from torcheval.metrics.classification import MulticlassRecall
from models import OnlineModel
import matplotlib.pyplot as plt

from torcheval.metrics.functional import multiclass_recall, multiclass_precision, multiclass_f1_score

In [10]:
plt.style.use("ggplot")

In [11]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [12]:
batch_size = 256
epochs = 12

In [13]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10 dataset split.
dataset_train = datasets.CIFAR10(
        root='data',
        train=True,
        download=True,
        transform=transform,
    )
dataset_train, dataset_val = random_split(dataset_train, [45000, 5000])
dataset_train, _ = random_split(dataset_train, [4500, 40500])
dataset_val, _ = random_split(dataset_val, [500, 4500])
dataset_test = datasets.CIFAR10(
        root='data',
        train=False,
        download=True,
        transform=transform,
    )
    # Create data loaders.
train_loader = DataLoader(
        dataset_train, 
        batch_size=batch_size,
        shuffle=True
    )
val_loader = DataLoader(
        dataset_val, 
        batch_size=batch_size,
        shuffle=True
    )
test_loader = DataLoader(
        dataset_test, 
        batch_size=batch_size,
        shuffle=True
    )

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# print(multiclass_f1_score(torch.tensor([1, 1, 1, 1]), torch.tensor([0, 1, 1, 1]), num_classes=2, average="micro").item())
# print(multiclass_precision(torch.tensor([1, 1, 1, 1]), torch.tensor([0, 1, 1, 1]), num_classes=2, average="micro").item())
# print(multiclass_recall(torch.tensor([1, 1, 1, 1]), torch.tensor([0, 1, 1, 1]), num_classes=2, average="micro").item())

### Baseline

In [15]:
model = resnet50()
model.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [16]:
# f1_metric = MulticlassF1Score(num_classes=10)
# p_metric = MulticlassPrecision(num_classes=10)
# r_metric = MulticlassRecall(num_classes=10)

train_loss, valid_loss = [], []
f1_train, f1_valid = [], []
p_train, p_valid = [], []
r_train, r_valid = [], []

# Start the training.
for epoch in range(epochs):
    model.train()
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")

    print('Training')
    train_epoch_loss = 0.0
    counter = 0
    train_f1 = 0.0
    train_p = 0.0
    train_r = 0.0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        # f1_metric.update(outputs, labels)
        # p_metric.update(outputs, labels)
        # r_metric.update(outputs, labels)
        # train_f1 += f1_metric.compute().item()
        # train_p += p_metric.compute().item()
        # train_r += r_metric.compute().item()
        train_f1 += multiclass_f1_score(outputs, labels, num_classes=10, average="macro").item()
        train_p += multiclass_precision(outputs, labels, num_classes=10, average="macro").item()
        train_r += multiclass_recall(outputs, labels, num_classes=10, average="macro").item()
        loss.backward()
        optimizer.step()
        # print statistics
        train_epoch_loss += loss.item()

    train_epoch_loss = train_epoch_loss / counter
    train_f1 = train_f1 / counter
    train_p = train_p / counter
    train_r = train_r / counter

    model.eval()
    print("Validation")
    valid_epoch_loss = 0
    counter = 0
    val_f1 = 0.0
    val_p = 0.0
    val_r = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader), total=len(val_loader)):
            counter += 1
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # forward + backward + optimize
            outputs = model(inputs)

            # f1_metric.update(outputs, labels)
            # p_metric.update(outputs, labels)
            # r_metric.update(outputs, labels)
            val_f1 += multiclass_f1_score(outputs, labels, num_classes=10, average="macro").item()
            val_p += multiclass_precision(outputs, labels, num_classes=10, average="macro").item()
            val_r += multiclass_recall(outputs, labels, num_classes=10, average="macro").item()
            loss = criterion(outputs, labels)
            # print statistics
            valid_epoch_loss += loss.item()

    valid_epoch_loss = valid_epoch_loss / counter
    val_f1 = val_f1 / counter
    val_p = val_p / counter
    val_r = val_r / counter

    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)

    f1_train.append(train_f1)
    f1_valid.append(val_f1)

    p_train.append(train_p)
    p_valid.append(val_p)

    r_train.append(train_r)
    r_valid.append(val_r)

    print(f"Training loss: {train_epoch_loss:.3f}, f1: {train_f1}, p: {train_p}, r: {train_r}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, f1: {val_f1}, p: {val_p}, r: {val_r}")
    print('-'*50)
        
print('TRAINING COMPLETE')

[INFO]: Epoch 1 of 12
Training


100%|██████████| 18/18 [00:01<00:00, 13.15it/s]


Validation


100%|██████████| 2/2 [00:00<00:00, 17.92it/s]


Training loss: 2.490, f1: 0.0953126907762554, p: 0.10864415475063854, r: 0.10919060930609703
Validation loss: 2.351, f1: 0.07910642400383949, p: 0.12404929473996162, r: 0.12152556329965591
--------------------------------------------------
[INFO]: Epoch 2 of 12
Training


...

Training loss: 1.824, f1: 0.3361483845445845, p: 0.3733382026354472, r: 0.35214420325226253
Validation loss: 2.143, f1: 0.22893569618463516, p: 0.23847205191850662, r: 0.23860371112823486
--------------------------------------------------
[INFO]: Epoch 12 of 12
Training


100%|██████████| 18/18 [00:01<00:00, 17.22it/s]


Validation


100%|██████████| 2/2 [00:00<00:00, 26.46it/s]

Training loss: 1.764, f1: 0.3517172535260518, p: 0.37306736409664154, r: 0.36469634705119663
Validation loss: 2.142, f1: 0.23503126949071884, p: 0.25170814990997314, r: 0.2553534358739853
--------------------------------------------------
TRAINING COMPLETE


In [17]:
torch.save(model.state_dict(), f"./weights/base")

### Pretrained feature extractor

In [18]:
model = OnlineModel().encoder
model.load_state_dict(torch.load("../BYOL/pretrained_feature_extractors/feature_extractor_20"))
model.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [19]:
# # Freeze all layers
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze last layer
# for param in model.fc.parameters():
#     param.requires_grad = True

In [20]:
train_loss_pretrained, valid_loss_pretrained = [], []
f1_train_pretrained, f1_valid_pretrained = [], []
p_train_pretrained, p_valid_pretrained = [], []
r_train_pretrained, r_valid_pretrained = [], []

# Start the training.
for epoch in range(epochs):
    model.train()
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")

    print('Training')
    train_epoch_loss = 0.0
    counter = 0
    train_f1 = 0.0
    train_p = 0.0
    train_r = 0.0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # f1_metric.update(outputs, labels)
        # p_metric.update(outputs, labels)
        # r_metric.update(outputs, labels)
        train_f1 += multiclass_f1_score(outputs, labels, num_classes=10, average="macro").item()
        train_p += multiclass_precision(outputs, labels, num_classes=10, average="macro").item()
        train_r += multiclass_recall(outputs, labels, num_classes=10, average="macro").item()
        loss.backward()
        optimizer.step()
        # print statistics
        train_epoch_loss += loss.item()

    train_epoch_loss = train_epoch_loss / counter
    train_f1 = train_f1 / counter
    train_p = train_p / counter
    train_r = train_r / counter

    model.eval()
    print("Validation")
    valid_epoch_loss = 0
    counter = 0
    val_f1 = 0.0
    val_p = 0.0
    val_r = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader), total=len(val_loader)):
            counter += 1
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            # f1_metric.update(outputs, labels)
            # p_metric.update(outputs, labels)
            # r_metric.update(outputs, labels)
            val_f1 += multiclass_f1_score(outputs, labels, num_classes=10, average="macro").item()
            val_p += multiclass_precision(outputs, labels, num_classes=10, average="macro").item()
            val_r += multiclass_recall(outputs, labels, num_classes=10, average="macro").item()
            loss = criterion(outputs, labels)
            # print statistics
            valid_epoch_loss += loss.item()

    valid_epoch_loss = valid_epoch_loss / counter
    val_f1 = val_f1 / counter
    val_p = val_p / counter
    val_r = val_r / counter

    train_loss_pretrained.append(train_epoch_loss)
    valid_loss_pretrained.append(valid_epoch_loss)

    f1_train_pretrained.append(train_f1)
    f1_valid_pretrained.append(val_f1)

    p_train_pretrained.append(train_p)
    p_valid_pretrained.append(val_p)

    r_train_pretrained.append(train_r)
    r_valid_pretrained.append(val_r)

    print(f"Training loss: {train_epoch_loss:.3f}, f1: {train_f1}, p: {train_p}, r: {train_r}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, f1: {val_f1}, p: {val_p}, r: {val_r}")
    print('-'*50)
        
print('TRAINING COMPLETE')


[INFO]: Epoch 1 of 12
Training


100%|██████████| 18/18 [00:04<00:00,  3.74it/s]


Validation


100%|██████████| 2/2 [00:00<00:00,  9.35it/s]


Training loss: 2.357, f1: 0.04024852791594134, p: 0.04881262000546687, r: 0.10775816233621703
Validation loss: 2.399, f1: 0.040535902604460716, p: 0.04894971288740635, r: 0.10632399842143059
--------------------------------------------------
[INFO]: Epoch 2 of 12
Training


...

Training loss: 2.309, f1: 0.0628015057494243, p: 0.07648795605119732, r: 0.13573211638463867
Validation loss: 2.357, f1: 0.05098233371973038, p: 0.06255516409873962, r: 0.11834509670734406
--------------------------------------------------
[INFO]: Epoch 12 of 12
Training


100%|██████████| 18/18 [00:04<00:00,  3.85it/s]


Validation


100%|██████████| 2/2 [00:00<00:00,  9.56it/s]

Training loss: 2.288, f1: 0.06886826517681281, p: 0.09209533677332932, r: 0.13297979409495989
Validation loss: 2.309, f1: 0.06492675840854645, p: 0.0791527982801199, r: 0.12506486102938652
--------------------------------------------------
TRAINING COMPLETE


In [21]:
torch.save(model.state_dict(), f"./weights/pretrained")

### Plots

In [22]:
plt.plot(f1_train_pretrained, label='f1_train_pretrained')
plt.plot(f1_valid_pretrained, label='f1_valid_pretrained')
plt.plot(f1_train, label='f1_train')
plt.plot(f1_valid, label='f1_valid')

plt.legend()
plt.savefig(f"./plots/f1", bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [23]:
plt.plot(p_train_pretrained, label='p_train_pretrained')
plt.plot(p_valid_pretrained, label='p_valid_pretrained')
plt.plot(p_train, label='p_train')
plt.plot(p_valid, label='p_valid')

plt.legend()
plt.savefig(f"./plots/precision", bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [24]:
plt.plot(r_train_pretrained, label='r_train_pretrained')
plt.plot(r_valid_pretrained, label='r_valid_pretrained')
plt.plot(r_train, label='r_train')
plt.plot(r_valid, label='r_valid')

plt.legend()
plt.savefig(f"./plots/recall", bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [25]:
plt.plot(train_loss_pretrained, label='train_loss_pretrained')
plt.plot(valid_loss_pretrained, label='train_loss_pretrained')
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss, label='train_loss')

plt.legend()
plt.savefig(f"./plots/loss", bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>